In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_tsvd import *
from parameters import *
import GPy
import scipy.stats

import matplotlib.pyplot as plt
np.random.seed(101)



### Importing the Tracer Data to Select the Optimisation set S

In [2]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


### Working subset of the data : set S

In [4]:
S = working_subset(data_df, loc_df, nbins = (25,25,25), threshold_sum = 10**-2 )

The remaining number of points is :  57725


### Import the Field of interest for computing the optimisation : TracerBackground / Pressure

In [5]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [6]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

  0%|          | 0/1 [00:00<?, ?it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [8]:
X = loc_df.values[:,:] #[I,:dim]
Z = data_df.values[:,:]#[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Z.shape

(100040, 989)

### Detrending Data : 
Plotting the data along each direction in order to see if there is a obvious trend. It seams that it is not the case. 

In [9]:
Z[S,:] = (Z[S,:]  - Z[S,:].mean(axis=1,keepdims=True))

### Sensor Optimisation with TSVD : 


Define the Sets for the optimisation

In [10]:
sets = define_sets(S)

Number of sensors to place : 

In [11]:
k = 5

Truncation parameteter for the TSVD : 

In [12]:
tau = 50

In [13]:
A_opt = {}

In [14]:
A_opt['lazy'] = sensor_loc_optimisation_lazy(k,Z, sets, tau)

Inner Loop:   0%|          | 62/57725 [02:48<42:11:34,  2.63s/it]

KeyboardInterrupt: 

In [ ]:
A_opt['naive'] = sensor_loc_optimisation_naive(k,Z, sets, tau)

In [ ]:
A_opt['lazy'] 

In [ ]:
A_opt['naive']